<a href="https://colab.research.google.com/github/royerlozano/CUADERNO-DE-ROYER/blob/main/RoyerLozanoPreg3Susti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://nyu-cds.github.io/python-gpu/02-cuda/

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-331-updates-uvm' for glob 'nvidi

In [ ]:
!pip install ipython-autotime

%load_ext autotime

time: 130 µs (started: 2021-03-15 15:43:05 +00:00)


In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include <cuda.h>
#include <cuda_runtime.h>


#define MAX_ERR 1e-6

#include <stdio.h>
#include <stdlib.h>
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#define N 1000 // tamano de los vectores
#define BLOCK 5 // tamano del bloque
// declaracion de funciones
// GLOBAL: funcion llamada desde el host y ejecutada en el device (kernel)
__global__ void suma(float* a, float* b, float* c)
{
int myID = threadIdx.x + blockDim.x * blockIdx.x;
// Solo trabajan N hilos
if (myID < N)
{
c[myID] = a[myID] + b[myID];
}
}
// MAIN: rutina principal ejecutada en el host
int main(int argc, char** argv)
{
// declaraciones
float* vector1, * vector2, * resultado;
float* dev_vector1, * dev_vector2, * dev_resultado;
 
// reserva en el host
vector1 = (float*)malloc(N * sizeof(float));
vector2 = (float*)malloc(N * sizeof(float));
resultado = (float*)malloc(N * sizeof(float));
// reserva en el device
cudaMalloc((void**)&dev_vector1, N * sizeof(float));
cudaMalloc((void**)&dev_vector2, N * sizeof(float));
cudaMalloc((void**)&dev_resultado, N * sizeof(float));
// inicializacion de vectores
for (int i = 0; i < N; i++)
{
vector1[i] = (float)rand() / RAND_MAX;
vector2[i] = (float)rand() / RAND_MAX;
}
// copia de datos hacia el device
cudaMemcpy(dev_vector1, vector1, N * sizeof(float), cudaMemcpyHostToDevice);
cudaMemcpy(dev_vector2, vector2, N * sizeof(float), cudaMemcpyHostToDevice);
// lanzamiento del kernel
// calculamos el numero de bloques necesario para un tamaño de bloque fijo
int nBloques = N / BLOCK;
if (N % BLOCK != 0)
{
nBloques = nBloques + 1;
}
int hilosB = BLOCK;
printf("Vector de %d elementos\n", N);
printf("Lanzamiento con %d bloques (%d hilos)\n", nBloques, nBloques * hilosB);
suma << < nBloques, hilosB >> > (dev_vector1, dev_vector2, dev_resultado);
// recogida de datos desde el device
cudaMemcpy(resultado, dev_resultado, N * sizeof(float), cudaMemcpyDeviceToHost);
// impresion de resultados
printf("> vector1:\n");
for (int i = 0; i < N; i++)
{
printf("%.2f ", vector1[i]);
}
printf("\n");
printf("> vector2:\n");
for (int i = 0; i < N; i++)
{
printf("%.2f ", vector2[i]);
}
printf("\n");
printf("> SUMA:\n");
for (int i = 0; i < N; i++)
{
printf("%.2f ", resultado[i]);
}
printf("\n");
// liberamos memoria en el device
cudaFree(dev_vector1);
cudaFree(dev_vector2);
cudaFree(dev_resultado);
// salida
printf("\npulsa INTRO para finalizar...");
fflush(stdin);
char tecla = getchar();
return 0;
}


Vector de 1000 elementos
Lanzamiento con 200 bloques (1000 hilos)
> vector1:
0.84 0.78 0.91 0.34 0.28 0.48 0.36 0.95 0.64 0.14 0.02 0.14 0.16 0.13 1.00 0.51 0.61 0.64 0.49 0.29 0.53 0.40 0.28 0.81 0.07 0.53 0.19 0.89 0.06 0.46 0.24 0.90 0.27 0.38 0.51 0.53 0.44 0.93 0.28 0.64 0.69 0.44 0.83 0.23 0.35 0.96 0.66 0.44 0.40 0.68 0.48 0.95 0.15 0.64 0.62 0.79 0.45 0.19 0.56 0.17 0.10 0.50 0.98 0.68 0.75 0.29 0.58 0.15 0.13 0.16 0.07 0.05 0.18 0.80 0.66 0.64 0.09 0.52 0.07 0.46 0.57 0.05 1.00 0.89 1.00 0.87 0.00 0.59 0.16 0.91 0.36 0.58 0.69 0.53 0.30 0.58 0.75 0.04 0.83 0.87 0.98 0.90 0.67 0.16 0.89 0.65 0.63 0.70 0.33 0.07 0.22 0.51 0.28 0.72 0.47 0.94 0.34 0.43 0.34 0.83 0.68 0.48 0.71 0.62 0.41 0.67 0.35 0.61 0.73 0.74 0.92 0.65 0.53 0.26 0.69 0.11 0.58 0.67 0.78 0.33 0.98 0.83 0.19 0.96 0.76 0.12 0.38 0.94 0.86 0.79 0.30 0.91 0.50 0.16 0.86 0.46 0.50 0.18 0.73 0.60 0.84 0.18 0.50 0.14 0.32 0.91 0.84 0.50 0.39 0.61 0.15 0.11 0.36 0.33 0.43 0.58 0.66 0.49 0.88 0.52 0.56 0.83 0.24 0.73 0.9